In [10]:
%cd examples

/Users/Chris/Documents/UWLing/altk/src/examples


In [11]:
from learn_quant.meaning import create_universe
from learn_quant.util import read_expressions

import numpy as np

from learn_quant.measures_vectorized import MonotonicityMeasurer

## Introduction

 First, let's familiarize ourselves with the classes used in this example, which are subclasses from classes in the ALTK package.

### QuantifierModel

In this example, we'd like to create a large number of quantifiers that are modeled by the class `QuantifierModel`. As stated in the `meaning` module, a `QuantifierModel`, every quantifier model is a triple ** <M, A, B> **, where M corresponds to all possible quantifier referents for a given communicative situation, A and B are differents sets of quantifier referents that correspond to the items of comparison in quantificational logic.

Let's begin by creating a simple QuantifierModel object. A QuantifierModel is initialized by defining a sequence of symbols that denote the set composition of each of `M`, `A`, and `B`. 

The definitions for each symbol are as follows:

`0 => in A`

`1 => in B`

`2 => in (A | B)`

`3 => in M - (A | B)`

`4 => not in (M | A | B)`


In [13]:
from learn_quant.quantifier import QuantifierModel

qm = QuantifierModel("10123221")

In [16]:
qm

QuantifierModel(name='10123221', M=frozenset({0, 1, 2, 3, 4, 5, 6, 7}), A=frozenset({1, 3, 5, 6}), B=frozenset({0, 2, 3, 5, 6, 7}))

Notice that all sets are typed as `frozenset`s, as this allows for hashing and checking of QuantifierModels in Meaning objects that is required for subsequent routines. 

### Importing a `QuantifierGrammar`

A QuantifierGrammar is a regular Grammar object, but allows for primitives representing integers to be added after a basic grammar has been loaded. This allows for integer primitives to be created on the fly during experiments of certain lengths (you usually would want to allow for primitives up until the length of the size of M, or all referents that are "in play"). 

In [23]:
from learn_quant.grammar import quantifiers_grammar

You can iterate through the grammar to see what rules it contains:

In [37]:
for rule in quantifiers_grammar:
    print(rule[0], ":", rule[1])

and : bool -> and(bool, bool)
or : bool -> or(bool, bool)
not : bool -> not(bool)
union : frozenset -> union(frozenset, frozenset)
intersection : frozenset -> intersection(frozenset, frozenset)
difference : frozenset -> difference(frozenset, frozenset)
index : frozenset -> index(int, frozenset)
cardinality : int -> cardinality(frozenset)
subset_eq : bool -> subset_eq(frozenset, frozenset)
equals : bool -> equals(int, int)
greater_than : bool -> greater_than(int, int)
A : frozenset -> A
B : frozenset -> B


To add primitives for integer indices, use the `add_indices_as_primitives` method on the `QuantifierGrammar` object by specifying either specific indices in a list, or an integer upper bound up until which primitive rules should be added:

In [39]:
from copy import deepcopy
new_grammar = deepcopy(quantifiers_grammar)
new_grammar.add_indices_as_primitives(["0", "1", "3", "4", "8", "9"], 2.0)
for rule in new_grammar:
    print(rule[0], ":", rule[1])

and : bool -> and(bool, bool)
or : bool -> or(bool, bool)
not : bool -> not(bool)
union : frozenset -> union(frozenset, frozenset)
intersection : frozenset -> intersection(frozenset, frozenset)
difference : frozenset -> difference(frozenset, frozenset)
index : frozenset -> index(int, frozenset)
cardinality : int -> cardinality(frozenset)
subset_eq : bool -> subset_eq(frozenset, frozenset)
equals : bool -> equals(int, int)
greater_than : bool -> greater_than(int, int)
A : frozenset -> A
B : frozenset -> B
0 : int -> 0
1 : int -> 1
3 : int -> 3
4 : int -> 4
8 : int -> 8
9 : int -> 9


In [56]:
new_grammar = deepcopy(quantifiers_grammar)
new_grammar.add_indices_as_primitives(4, 2.0)
for rule in new_grammar:
    print(rule[0], ":", rule[1])

and : bool -> and(bool, bool)
or : bool -> or(bool, bool)
not : bool -> not(bool)
union : frozenset -> union(frozenset, frozenset)
intersection : frozenset -> intersection(frozenset, frozenset)
difference : frozenset -> difference(frozenset, frozenset)
index : frozenset -> index(int, frozenset)
cardinality : int -> cardinality(frozenset)
subset_eq : bool -> subset_eq(frozenset, frozenset)
equals : bool -> equals(int, int)
greater_than : bool -> greater_than(int, int)
A : frozenset -> A
B : frozenset -> B
0 : int -> 0
1 : int -> 1
2 : int -> 2
3 : int -> 3
4 : int -> 4


The second argument defines a default weight to add to the integer primitives rules that are relevant when generating a universe with a `Grammar`.

In [54]:
for rule in new_grammar:
    print(rule[1].name, ":\t\t", rule[1].weight)

and :		 1.0
or :		 1.0
not :		 1.0
union :		 1.0
intersection :		 1.0
difference :		 1.0
index :		 1.0
cardinality :		 1.0
subset_eq :		 1.0
equals :		 1.0
greater_than :		 1.0
A :		 10.0
B :		 10.0
0 :		 2.0
1 :		 2.0
2 :		 2.0
3 :		 2.0


### Define a universe of referents.

In this example, the function "create_universe" creates QuantifierModels, 

In [60]:

quantifiers_universe = create_universe(M_SIZE=3,X_SIZE=4)

We created a universe with the number of indices in generated `QuantifierModel`s having 4 indices total, with up to 3 of those indices being considered for pertinence in A or B during the generative process. Therefor, in this example, `['1', '2', '0', '0']` would not be valid, since `M_SIZE` is only 3 and not 4. On the other hand, `['4', '2', '0', '0']` is OK, since the first index is in `X` but not `M`.

Let's enumerate expressions that could be created with the Language of Thought described in the `QuantifierGrammar` we have previously defined.

We'll enumerate expressions up to a depth of 4. Higher depth values allow for more complex expressions that depend on a greater number of rules.

In [61]:
from learn_quant.scripts.generate_expressions import enumerate_quantifiers
expressions_by_meaning = enumerate_quantifiers(4, quantifiers_universe, quantifiers_grammar)

In [63]:
print("The size of the universe is {}".format(len(quantifiers_universe)))

The size of the universe is 256


Access the referents by refering to the `referents` property of the `QuantifierUniverse` object

In [67]:
quantifiers_universe.referents[0:5]

[QuantifierModel(name='4031', M=frozenset({1, 2, 3}), A=frozenset({1}), B=frozenset({3})),
 QuantifierModel(name='1242', M=frozenset({0, 1, 3}), A=frozenset({1, 3}), B=frozenset({0, 1, 3})),
 QuantifierModel(name='4023', M=frozenset({1, 2, 3}), A=frozenset({1, 2}), B=frozenset({2})),
 QuantifierModel(name='1410', M=frozenset({0, 2, 3}), A=frozenset({3}), B=frozenset({0, 2})),
 QuantifierModel(name='3241', M=frozenset({0, 1, 3}), A=frozenset({1}), B=frozenset({1, 3}))]

You can access sizes of `X` and `M` in the QuantifierUniverse object:

In [72]:
print(quantifiers_universe.x_size)
print(quantifiers_universe.m_size)

4
3


In [ ]:
from learn_quant.scripts.generate_expressions import save_quantifiers
